In [80]:
from dbfread import DBF
import pandas as pd

In [75]:
dadinhos = pd.read_csv(rf'..\2015.csv', sep=';')
# dadinhos = dadinhos.drop(columns=['Unnamed: 0'], axis=1)
dadinhos

C:\Users\isabe\AppData\Local\Temp\ipykernel_4716\3607087470.py:1: DtypeWarning: Columns (96,97,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  dadinhos = pd.read_csv(rf'..\2015.csv', sep=';')


,UF_ZI,ANO_CMPT,MES_CMPT,ESPEC,CGC_HOSP,N_AIH,IDENT,CEP,MUNIC_RES,NASC,...,DIAGSEC9,TPDISEC1,TPDISEC2,TPDISEC3,TPDISEC4,TPDISEC5,TPDISEC6,TPDISEC7,TPDISEC8,TPDISEC9
0,120000,2015,1,1,6.360294e+13,1215100018110,1,69900970,120040,19691208,...,NaN,1,0,0,0,0,0,0,0,0
1,120000,2015,1,1,6.360294e+13,1215100018198,1,69900970,120040,19730513,...,NaN,1,0,0,0,0,0,0,0,0
2,120000,2015,1,1,6.360294e+13,1215100018341,1,69921000,120080,19530312,...,NaN,1,0,0,0,0,0,0,0,0
3,120000,2015,1,1,6.360294e+13,1215100018363,1,69900970,120040,19910618,...,NaN,1,0,0,0,0,0,0,0,0
4,120000,2015,1,1,6.360294e+13,1215100018385,1,69960000,120030,19950619,...,NaN,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,110000,2015,2,1,6.074262e+13,1115500004542,1,76900100,110012,19460501,...,NaN,0,0,0,0,0,0,0,0,0
1048571,110000,2015,2,1,6.074262e+13,1115500004553,1,76868000,110013,19560826,...,NaN,0,0,0,0,0,0,0,0,0
1048572,110000,2015,2,1,6.074262e+13,1115500004510,1,76801000,110020,19510602,...,NaN,0,0,0,0,0,0,0,0,0
1048573,110000,2015,2,1,4.287520e+12,1115100053254,1,76801000,110020,19711106,...,NaN,0,0,0,0,0,0,0,0,0


In [ ]:
data = dadinhos.copy()

# Remover algumas colunas específicas
data = data.drop(columns=['NASC', 'CEP', 'MES_CMPT', 'NATUREZA', 'TPDISEC5'])

# Trocar os valores de 3 para 0 em sexo
data['SEXO'] = data['SEXO'].replace(3, 0)

# Remover idades zeradas
mask = data['IDADE'] != 0
data = data.drop(data[mask == False].index)

# Remover colunas inteiras vazias
data = data.dropna(how='all', axis=1)

# Remover colunas com todos os valores iguais
colunas_com_valores_unicos = data.nunique() == 1
if colunas_com_valores_unicos.any():
    data = data.drop(data.columns[colunas_com_valores_unicos], axis=1)

data

,UF_ZI,ESPEC,CGC_HOSP,N_AIH,MUNIC_RES,SEXO,UTI_MES_TO,MARCA_UTI,UTI_INT_TO,DIAR_ACOM,...,VAL_UCI,MARCA_UCI,DIAGSEC1,DIAGSEC2,DIAGSEC3,DIAGSEC4,TPDISEC1,TPDISEC2,TPDISEC3,TPDISEC4
0,120000,1,6.360294e+13,1215100018110,120040,0,0,0,0,0,...,"0,00",0,W199,NaN,NaN,NaN,1,0,0,0
1,120000,1,6.360294e+13,1215100018198,120040,1,0,0,0,0,...,"0,00",0,X599,NaN,NaN,NaN,1,0,0,0
2,120000,1,6.360294e+13,1215100018341,120080,0,0,0,0,4,...,"0,00",0,X599,NaN,NaN,NaN,1,0,0,0
3,120000,1,6.360294e+13,1215100018363,120040,0,0,0,0,0,...,"0,00",0,X599,NaN,NaN,NaN,1,0,0,0
4,120000,1,6.360294e+13,1215100018385,120030,1,0,0,0,0,...,"0,00",0,X599,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,110000,1,6.074262e+13,1115500004542,110012,1,0,0,0,1,...,"0,00",0,NaN,NaN,NaN,NaN,0,0,0,0
1048571,110000,1,6.074262e+13,1115500004553,110013,0,0,0,0,0,...,"0,00",0,NaN,NaN,NaN,NaN,0,0,0,0
1048572,110000,1,6.074262e+13,1115500004510,110020,1,0,0,0,1,...,"0,00",0,NaN,NaN,NaN,NaN,0,0,0,0
1048573,110000,1,4.287520e+12,1115100053254,110020,1,0,0,0,0,...,"0,00",0,NaN,NaN,NaN,NaN,0,0,0,0


In [ ]:
def encontrar_indice_true(lista):
  """
  Encontra e imprime o índice da primeira linha com o valor True em uma lista.

  Args:
    lista: Uma lista contendo valores booleanos.

  Returns:
    None. Imprime o índice (se encontrado) ou uma mensagem indicando que não foi encontrado.
  """
  for indice, valor in enumerate(lista):
    if valor == True:
      print(colunas_com_valores_unicos.index[indice])


# Exemplo de uso
encontrar_indice_true(colunas_com_valores_unicos)

ANO_CMPT
IDENT
UTI_MES_IN
UTI_MES_AN
UTI_MES_AL
UTI_INT_IN
UTI_INT_AN
UTI_INT_AL
VAL_SADT
VAL_RN
VAL_ACOMP
VAL_ORTP
VAL_SANGUE
VAL_SADTSR
VAL_TRANSP
VAL_OBSANG
VAL_PED1AC
DIAG_SECUN
RUBRICA
CAR_INT
TOT_PT_SP
SEQ_AIH5
CID_ASSO
CID_MORTE
DIAGSEC5
TPDISEC6
TPDISEC7
TPDISEC8
TPDISEC9


In [ ]:
data.to_csv(rf'..\2015_python.csv')